In [ ]:
using Pipe
using MLStyle

In [ ]:
@data RPSMove begin
  Rock()
  Paper()
  Scissors()
end

In [ ]:
movescore(move::RPSMove) =
  @match (move) begin
    Rock() => 1
    Paper() => 2
    Scissors() => 3
  end

roundOutcome(a::RPSMove, b::RPSMove) =
  @match (a, b) begin
    (Rock(), Paper()) => 6
    (Scissors(), Rock()) => 6
    (Paper(), Scissors()) => 6
    (a, b) => a == b ? 3 : 0
  end

roundScore(a::RPSMove, b::RPSMove) =
  begin
    return roundOutcome(a, b) + movescore(b)
  end

In [ ]:
beat(move::RPSMove) =
  @match (move) begin
    Rock() => Paper()
    Paper() => Scissors()
    Scissors() => Rock()
  end

lose(move::RPSMove) =
  @match (move) begin
    Rock() => Scissors()
    Paper() => Rock()
    Scissors() => Paper()
  end

In [ ]:

parseOponent(encryptedMove::AbstractString) =
  @match (encryptedMove) begin
    "A" => Rock()
    "B" => Paper()
    "C" => Scissors()
  end

parseSelf(encryptedMove::AbstractString) =
  @match (encryptedMove) begin
    "X" => Rock()
    "Y" => Paper()
    "Z" => Scissors()
  end

parseInputLine1(line::String) =
  begin
    encryptedMoves = split(line)
    return (parseOponent(first(encryptedMoves)), parseSelf(last(encryptedMoves)))
  end

In [ ]:
parseSelfMatchOpponent(encryptedMove::AbstractString, opponentMove::RPSMove) =
@match (encryptedMove) begin
  "X" => lose(opponentMove)
  "Y" => identity(opponentMove)
  "Z" => beat(opponentMove)
end

parseSelfMatchOpponentFunction(encryptedMove::AbstractString) =
@match (encryptedMove) begin
  "X" => lose
  "Y" => identity
  "Z" => beat
end

parseInputLine2(line::String) =
  begin
    encryptedMoves = split(line)
    opMove = parseOponent(first(encryptedMoves))
    # myMove = parseSelfMatchOpponent(last(encryptedMoves), opMove)
    myMove = parseSelfMatchOpponentFunction(last(encryptedMoves))(opMove)
    return (opMove, myMove)
  end

In [ ]:
# moveChoices = Vector{Tuple{RPSMove, RPSMove}}[]
moveChoices = Vector{Tuple{RPSMove,RPSMove}}()
for moveChoice in eachline("input.txt")
  @pipe moveChoice |> parseInputLine2 |> push!(moveChoices, _)
end

In [ ]:
map(Base.splat(roundScore), moveChoices) |> sum